# Decorators and Closures

## Decorators
A decorator is a callable that takes a function as argument and returns another one. It may perform some processing on the given function and return it, or return another one.

They are executed immediately when a module is loaded. a decorator is run as many times as it decorates a function---if a decorator is used 3 times in a module for 3 different functions, it will run 3 times, one for each, when the module is imported/ran. But the decorated functions will only run when invoked (like normal functions)

In [4]:
# Swap the given function with another one
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


## Variable scope rules
When Python compiles the body of a function, it will take of variables assigned within the body and decide that they're local, so evn if they are available in the global context, the next code would fail. When Python compiles this function, it decides that `b` is a local variable because it is assigned within the function.

This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local. This is much better than the behavior of JavaScript, which does not require variable declarations either, but if you do forget to declare that a variable is local (with var), you may clobber a global variable without knowing.

In [11]:
b = 9
def foo(a):
    print(a)
    print(b)
    b = 6

try:
    foo(3)
except UnboundLocalError:
    print('Error!')
    
# In order to have a function treat a variable as global even
# if it is defined within the function's body, use global
def foo(a):
    global b
    print(a)
    print(b)
    b = 6

try:
    foo(3)
except UnboundLocalError:
    print('Error!')

3
Error!
3
9


In [18]:
# Assigning to a variable without affecting one defined elsewhere
import pandas as pd

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': ['A', 'B', 'B']})

def some_processing(df):
    df = df.groupby('c').mean()
    print(df)
    
some_processing(df)
print(df)

     a    b
c          
A  1.0  4.0
B  2.5  5.5
   a  b  c
0  1  4  A
1  2  5  B
2  3  6  B


In [20]:
import pandas as pd

df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': ['A', 'B', 'B']})

def some_processing(df):
    df = df.copy()
    df['d'] = [5, 6, 7]
    print(df)
    
some_processing(df)
print(df)

   a  b  c  d
0  1  4  A  5
1  2  5  B  6
2  3  6  B  7
   a  b  c
0  1  4  A
1  2  5  B
2  3  6  B


## Closures
Functions defined within the scope of another function---a function that returns another function. function in the inside scope has access to all local variables defined in the outer scope. 

In [5]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        return sum(series) / len(series)
    
    return averager

avg = make_averager()

print(avg(10), avg(11), avg(12))
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

10.0 10.5 11.0
(<cell at 0x00000177A2351498: list object at 0x00000177A2362148>,)
[10, 11, 12]


### Free Variables
Within averager, `series` is a *free variable*---a variable that is not bound in the local scope. The binding for `series` is kept in the `__closure__` attribute of the `avg` function. Each item in this attr is called a cell and corresponds to a *freevar*, and they have an attribute called `cell_contents` where the actual values are stored.

So put another way, closures are functions that retain the bindings of the free variables that exist in the scope when the function was defined, so they can be used later when the function is invoked and the scope is no longer available.

### Nonlocal declaration
In the previous example, series (a mutable list) is called and updated within the inner function, and remains as a free variable. But in this version:

```python
def make_averager():
    total = 0
    count = 0
    def averager(new_value):
        count = count + 1
        total = total + new_value
        return total / count 
    return averager
```

It wouldn't work because count and total are float/int, which are immutable, and when assigning to them `count = count + 1` python interprets them as a new assignment and decides they are local variables, instead of free variables. In order for this to work, we must tell python that these are free variables by adding the following line:

```python
# ...
def averager(new_value):
    # count and total are not local variables
    nonlocal count, total
    count = count + 1
    total = total + 1
    # ...
```

## Decorator implementation

In [38]:
# Example: decorator that times function
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}] {name}({arg_str}) -> {result}')
        return result
    return clocked
        

In [39]:
@clock
def foo(seconds):
    time.sleep(seconds)
    
foo(0.5)
print(foo.__name__)

[0.49876418] foo(0.5) -> None
foo


### Decorators in the standard library

#### `lru_cache`


Saves the results of previous invocations of a function, avoiding repeat computations on previously used arguments.

LRU stands for 'Least Recently Used', because it stores the most recent used results for given arguments.

It accepts 2 arguments:
- `maxsize`: `int`. Number of results to store. Old ones are dropped out the end.
- `typed`: `bool`. 

Because `lru_cache` will store results for each argument in a dictionary, arguments to the function itself must be hashable.


In [46]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(6)

[0.00000038] fibonacci(0) -> 0
[0.00000113] fibonacci(1) -> 1
[0.00030546] fibonacci(2) -> 1
[0.00000113] fibonacci(1) -> 1
[0.00000113] fibonacci(0) -> 0
[0.00000113] fibonacci(1) -> 1
[0.00010044] fibonacci(2) -> 1
[0.00019483] fibonacci(3) -> 2
[0.00059431] fibonacci(4) -> 3
[0.00000076] fibonacci(1) -> 1
[0.00000302] fibonacci(0) -> 0
[0.00000076] fibonacci(1) -> 1
[0.00008760] fibonacci(2) -> 1
[0.00019106] fibonacci(3) -> 2
[0.00000076] fibonacci(0) -> 0
[0.00000113] fibonacci(1) -> 1
[0.00008533] fibonacci(2) -> 1
[0.00000076] fibonacci(1) -> 1
[0.00000076] fibonacci(0) -> 0
[0.00000076] fibonacci(1) -> 1
[0.00005853] fibonacci(2) -> 1
[0.00011101] fibonacci(3) -> 2
[0.00025222] fibonacci(4) -> 3
[0.00050973] fibonacci(5) -> 5
[0.00127320] fibonacci(6) -> 8


8

In [47]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(6)

[0.00000038] fibonacci(0) -> 0
[0.00000076] fibonacci(1) -> 1
[0.00028658] fibonacci(2) -> 1
[0.00000113] fibonacci(3) -> 2
[0.00032472] fibonacci(4) -> 3
[0.00000151] fibonacci(5) -> 5
[0.00050634] fibonacci(6) -> 8


8

In [48]:
fibonacci(30)

[0.00000076] fibonacci(7) -> 13
[0.00026242] fibonacci(8) -> 21
[0.00000113] fibonacci(9) -> 34
[0.00030924] fibonacci(10) -> 55
[0.00000151] fibonacci(11) -> 89
[0.00036701] fibonacci(12) -> 144
[0.00000113] fibonacci(13) -> 233
[0.00040628] fibonacci(14) -> 377
[0.00000151] fibonacci(15) -> 610
[0.00045914] fibonacci(16) -> 987
[0.00000113] fibonacci(17) -> 1597
[0.00049992] fibonacci(18) -> 2584
[0.00000113] fibonacci(19) -> 4181
[0.00055731] fibonacci(20) -> 6765
[0.00000113] fibonacci(21) -> 10946
[0.00061432] fibonacci(22) -> 17711
[0.00000076] fibonacci(23) -> 28657
[0.00064982] fibonacci(24) -> 46368
[0.00000189] fibonacci(25) -> 75025
[0.00081935] fibonacci(26) -> 121393
[0.00000227] fibonacci(27) -> 196418
[0.00091828] fibonacci(28) -> 317811
[0.00000453] fibonacci(29) -> 514229
[0.00101569] fibonacci(30) -> 832040


832040

#### `singledispatch`

Used to create a generic dispatcher function that dispatches to specialized functions depending on the type of the first argument. These specialized functions can be registered anywhere in the system, in any module.

In [50]:
# Turn python objects into html representations
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    """ Main generic function """
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

# Start defining specialized functions to handle specific types
# Their names don't matter
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f"<p>{content}</p>"

@htmlize.register(numbers.Integral)
def _(n):
    return "<pre>{0} (0x{0:x})</pre>".format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    content = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return f'<ul>\n<li>{content}</li>\n</ul>'

In [53]:
print(htmlize(['My Pony', [15, 255, 343], {'Name': 'Juan', 'Last': 'Aguirre'}]))

<ul>
<li><p>My Pony</p></li>
<li><ul>
<li><pre>15 (0xf)</pre></li>
<li><pre>255 (0xff)</pre></li>
<li><pre>343 (0x157)</pre></li>
</ul></li>
<li><pre>{&#x27;Name&#x27;: &#x27;Juan&#x27;, &#x27;Last&#x27;: &#x27;Aguirre&#x27;}</pre></li>
</ul>
